In [ ]:
!pip install -q sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from __future__ import absolute_import,division,print_function,unicode_literals
import numpy as np #allows u to do multi dimensional calculations like vector operations, matrices, matix addition nd subtractiomn also represent data in easier form
import pandas as pd # allows us to manipulate data,load in datasets, visualize datasets, cut off rows from datasets  its a data analytic too
import matplotlib.pyplot as plt # allows us to visualize graphs and charts
from IPython.display import clear_output #as the name implies
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf


In [ ]:
#training dataset
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') #training dataset link
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') #testing dataset link
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')


In [ ]:
CATEGORICAL_COLUMNS = ['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']
NUMERICAL_COLUMNS = ['age','fare']
feature_columns =[]
for feature_name in CATEGORICAL_COLUMNS:
  volcabulary = dftrain[feature_name].unique() #this gets a list of all unique values from a given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name,volcabulary))


for feature_name in NUMERICAL_COLUMNS:
   feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print (feature_columns)





Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

In [ ]:
#the input function :this is created to input dataset that will be fed into the model to be created
def make_input_fn(data_df, label_df, no_epochs=10, shuffle=True, batch_size=32):
  def input_function(): #inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df)) # this create tf.data.Dataset with data and its label
    if shuffle:
      ds = ds.shuffle(1000)#randomly shuffles the data
    ds = ds.batch(batch_size).repeat(no_epochs)
    return ds
  return input_function
train_input_function = make_input_fn(dftrain,y_train) #here we call the input function that was returned tous in the defined
                                                      #function to get a dataset to feed to the model

eval_input_fn = make_input_fn(dfeval,y_eval,no_epochs=1,shuffle=False)

#CREATING A MODEL


linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

#TRAINING THE MODEL
linear_est.train(train_input_function) #train
result = linear_est.evaluate(eval_input_fn) #get model statistics
clear_output()
print(result)
print(result['accuracy'])

{'accuracy': 0.7537879, 'accuracy_baseline': 0.625, 'auc': 0.83033365, 'auc_precision_recall': 0.77246106, 'average_loss': 0.5172491, 'label/mean': 0.375, 'loss': 0.5137698, 'precision': 0.64166665, 'prediction/mean': 0.47010866, 'recall': 0.7777778, 'global_step': 200}
0.7537879


In [ ]:
#to predict survival
result = list(linear_est.predict(eval_input_fn))
clear_output()
print(dfeval.loc[200])
print (y_eval[200])
print(result[200]['probabilities'])
for results in result:
  print(results)


#classifications of datasets


sex                        male
age                        25.0
n_siblings_spouses            0
parch                         0
fare                      7.225
class                     Third
deck                    unknown
embark_town           Cherbourg
alone                         y
Name: 200, dtype: object
0
[0.7898811  0.21011887]
{'logits': array([-2.0694563], dtype=float32), 'logistic': array([0.11210115], dtype=float32), 'probabilities': array([0.8878988 , 0.11210114], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1], dtype=int32), 'all_classes': array([b'0', b'1'], dtype=object)}
{'logits': array([0.6122015], dtype=float32), 'logistic': array([0.6484428], dtype=float32), 'probabilities': array([0.3515572, 0.6484428], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1], dtype=int32), 'all_classes': array([b'0', b'1'], dtype=object)}
{'logits': array([1.78846

In [ ]:
dftrain.head()

In [ ]:
print (dftrain.loc[200],y_train[200])

In [ ]:
dftrain.describe()

In [ ]:
dftrain.shape

In [ ]:
y_train.head()

In [ ]:
dftrain['class'].hist(bins=20)

In [ ]:
dftrain.sex.value_counts().plot(kind='barh')

In [ ]:
dftrain['class'].value_counts().plot(kind='bar')

In [ ]:
pd.concat([dftrain,y_train],axis=1).groupby ('sex').survived.mean().plot(kind='bar').set_xlabel('%survive')